In [21]:
import ipeadatapy as ipea
import pandas as pd
import numpy as np

In [3]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9602,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9603,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9604,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9605,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais
9606,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas


In [4]:
series.loc[series.NAME == "IGP-DI"]

,CODE,NAME
4845,IGP_IGPDIG,IGP-DI
4875,IGP12_IGPDIG12,IGP-DI


In [12]:
df = (
    pd.read_csv("deflator.csv")
    .rename({"DPCERG3A086NBEA": "deflator"}, axis = 1)
    .assign(DATE = lambda _: pd.to_datetime(_.DATE))
)

In [14]:
bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV"),
    "base_monetaria": ipea.describe("BM_M0FN"),
    "inflacao": ipea.describe("IGP_IGPDIG")
}


for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]
    bases[base]["dataframe"] = (
        ipea.timeseries(bases[base]["Code"]).reset_index()
        .rename({f"VALUE ({bases[base]['Measure']})": "VALUE"}, axis = 1)
        .assign(
            nome = base,
            unidade = bases[base]['Unit'], 
            medida = bases[base]['Measure'])
    )


df = pd.concat([base["dataframe"] for base in bases.values()]).merge(df, how = "left")


In [30]:
(df
 .query("DATE.dt.year > 1950 and DATE.dt.year < 2000")
 .assign(
    VALUE = lambda _: np.where(_.medida == "US$", _.VALUE / _.deflator, _.VALUE),
    nome = lambda _: np.where(_.medida == "US$", _.nome + "_deflacionado", _.nome) + "_" + _.medida
)
 .pivot_table(
    index = "YEAR",
    columns = "nome",
    values = "VALUE"
).reset_index()
 .rename_axis("", axis = 1)
)

,YEAR,base_monetaria_R$,exportacao_deflacionado_US$,importacao_deflacionado_US$,inflacao_(% a.a.)
0,1951,1.589091e-11,124.973649,121.850512,12.337655
1,1952,1.869091e-11,98.173555,119.089650,12.716743
2,1953,2.214545e-11,105.180799,78.209976,20.512819
3,1954,2.690909e-11,105.829787,95.858992,25.864923
4,1955,3.192727e-11,96.061420,74.499865,12.145735
5,1956,3.745455e-11,98.072199,71.109258,24.548748
6,1957,5.003636e-11,89.377457,82.549004,6.956522
7,1958,5.887273e-11,77.988769,73.859769,24.390236
8,1959,8.105455e-11,79.251298,74.789812,39.433562
9,1960,1.096364e-10,77.171218,78.632687,30.468678
